In [1]:
import sys
import os
sys.path.append('..')
import dfx.datasets as datasets
import dfx.grain as dfx

# Diff
View row, column and value differences between two dataframes

In [9]:
c = datasets.checks.copy()
c = c.set_index(['employee_id', 'pay_date'])
c.loc[('24543', '2019-02-01'), 'state'] = 'CT'
df1 = c

c = datasets.checks.copy()
c.loc[c.index[c.pay_date=='2019-03-01'], 'pay_date'] = '2019-04-01'
c = c.set_index(['employee_id', 'pay_date'])
c.loc[('12345', '2019-01-01'), 'tax_rate'] = .35
c = c.sort_values('manager_id')
c['new_col'] = 'xyz'
df2 = c

In [10]:
# Generate the differences
diff = ops.DiffDf(df1, df2)

In [11]:
# rows added
diff.rows_added.head()

,,region,state,salary,company,manager_id,tax_rate,new_col
employee_id,pay_date,,,,,,,
00000,2019-04-01,west,NM,420,Acme,98765,0.28,xyz
12345,2019-04-01,east,NY,100,Acme,36363,0.28,xyz
24543,2019-04-01,east,NY,110,Acme,36363,0.28,xyz
36363,2019-04-01,east,NC,120,Acme,76576,0.28,xyz
48436,2019-04-01,east,NC,90,Acme,36363,0.28,xyz


In [12]:
# rows removed
diff.rows_removed.head()

,,region,state,salary,company,manager_id,tax_rate
employee_id,pay_date,,,,,,
00000,2019-03-01,west,NM,420,Acme,98765,0.28
12345,2019-03-01,east,NY,100,Acme,36363,0.28
24543,2019-03-01,east,NY,110,Acme,36363,0.28
36363,2019-03-01,east,NC,120,Acme,76576,0.28
48436,2019-03-01,east,NC,90,Acme,36363,0.28


In [13]:
# counts of value changes by column
diff.col_diff_counts

region        0
state         1
salary        0
company       0
manager_id    2
tax_rate      1
Name: diff_counts, dtype: int64

In [14]:
# see rows with side-by-side value changes
diff.df.head()

_merge state_1 state_2 manager_id_1 manager_id_2  \
employee_id pay_date                                                           
00000       2019-01-01        both      NM      NM        98765        98765   
            2019-02-01        both      NM      NM        98765        98765   
            2019-03-01   left_only      NM     NaN        98765          NaN   
            2019-04-01  right_only     NaN      NM          NaN        98765   
12345       2019-01-01        both      NY      NY        36363        36363   

                        tax_rate_1  tax_rate_2 region  salary company  \
employee_id pay_date                                                    
00000       2019-01-01        0.21        0.21   west   420.0    Acme   
            2019-02-01        0.21        0.21   west   420.0    Acme   
            2019-03-01        0.28         NaN   west   420.0    Acme   
            2019-04-01         NaN        0.28    NaN     NaN     NaN   
12345       2019-01-01        0.21        0.35   east   100.0    Acme   

                       new_col_2  
employee_id pay_date              
00000       2019-01-01       xyz  
            2019-02-01       xyz  
            2019-03-01       NaN  
            2019-04-01       xyz  
12345       2019-01-01       xyz

In [15]:
# see rows with a specific column change
diff.changed('tax_rate')

,,_merge,state_1,state_2,manager_id_1,manager_id_2,tax_rate_1,tax_rate_2,region,salary,company,new_col_2
employee_id,pay_date,,,,,,,,,,,
12345,2019-01-01,both,NY,NY,36363,36363,0.21,0.35,east,100.0,Acme,xyz
